In [1]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import random
import time

# Define parameters
batch_size = 32
epochs = 29
image_size = (299, 299)  # Image size expected by Xception

np.random.seed(20)
random.seed(20)
tf.random.set_seed(20)

# Define data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    r'C:\dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # use 'categorical' for multiple classes
    subset='training',
    shuffle=True  # Ensure data is shuffled
)

validation_generator = train_datagen.flow_from_directory(
    r'C:\dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # use 'categorical' for multiple classes
    subset='validation',
    shuffle=False  # No need to shuffle validation data
)

# Load Xception pre-trained on ImageNet
base_model = Xception(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  # Add more dense layers if needed
predictions = Dense(11, activation='softmax')(x)  # Adjust output neurons based on your problem (11 classes)

# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # use 'categorical_crossentropy' for multiple classes
              metrics=['accuracy'])

# Train the model with verbose logging and time callback
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose=1  # Ensure verbose output to monitor progress
)
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


Found 5505 images belonging to 11 classes.
Found 1372 images belonging to 11 classes.



Epoch 1/29


172/172 [==============================] - 1206s 7s/step - loss: 0.7228 - accuracy: 0.7643 - val_loss: 0.4602 - val_accuracy: 0.8460
Epoch 2/29
172/172 [==============================] - 1132s 7s/step - loss: 0.3747 - accuracy: 0.8714 - val_loss: 0.4717 - val_accuracy: 0.8438
Epoch 3/29
172/172 [==============================] - 1129s 7s/step - loss: 0.2766 - accuracy: 0.9059 - val_loss: 0.4685 - val_accuracy: 0.8497
Epoch 4/29
172/172 [==============================] - 1122s 7s/step - loss: 0.2088 - accuracy: 0.9340 - val_loss: 0.4696 - val_accuracy: 0.8557
Epoch 5/29
172/172 [==============================] - 1121s 7s/step - loss: 0.1576 - accuracy: 0.9540 - val_loss: 0.4598 - val_accuracy: 0.8601
Epoch 6/29
172/172 [==============================] - 1694s 10s/step - loss: 0.1339 - accuracy: 0.9560 - val_loss: 0.4514 - val_accuracy: 0.8638
Epoch 7/29
172/172 [=======================

In [2]:
# Save the model
model.save('Xception_model.h5')
#save history
import pickle
with open('Xception_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

C:\Users\s\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
